# Implementación del algoritmo PCKY
## Librerías a utilizar

Para la implementación del algoritmo CKY solo serán necesarias tres paquetes adicionales, el primero permite seleccionar un elemento de una lista a partir del índice, el segundo mostrará de manera más clara los resultados de los árboles y finalmente, pandas que permitirá mostrar la matriz CKY en forma tabular, como un dataframe. 

In [ ]:
from operator import itemgetter
from pprint import pprint
import pandas as pd

## Datos de entrada

La variable `grammar_file` que aparece a continuación emula las reglas gramaticales escritas en un archivo de texto. En este caso se han escrito las reglas propias del ejercicio.

In [ ]:
file = open("ds/grammar.txt",'r')
grammar_file = file.read()

La función `read_rules()` recibe como argumento un texto con reglas gramaticales y probabilidades para convierlas en tres diccionarios que permiten buscar los datos necesarios para completar el algoritmo CKY y PCKY.

Se crea el lexicón, de forma común, en lingüística se dice que un elemento del lexicón en la entrada de un diccionario, entonces se define como tal en el código.

Se expresan las reglas gramaticales y lexicas en forma de tuplas como claves de una probabilidad que se usará para obtener las probabilidades de cada árbol obtenido por el algoritmo. Se eligen tuplas porque son fáciles de manejar, similar a los lenguajes Lisp.

In [ ]:
def read_rules(grammar_file):
    '''
    This function read grammar rules writen in a text variable or a file.

    The text file contain grammar rules and probabilities writen like:

    S -> NP VP 0.800

    In grammar rules dictionary, parents indexed by children, like this:

    {('Det', 'Nominal'): ['NP'],
     ('NP', 'VP'): ['S'],
     ('Verb', 'PP'): ['VP']}

        Parameters:
            grammar_file (str): Text with grammar rules
                                                                  
        Returns:
            grammar_rules (dict): Dictionary with the rules indexed by children
            lexicon_rules (dict): Dictionary with terminal simbols
            probabilities (dict): Dictionary with grammar and lexical rules asociated to it's probability
    '''    
    gr_temp_rules = list()
    lexicon_rules = dict()
    probabilities = dict()
    grammar_rules = dict()
    for line in grammar_file.strip().split("\n"):
        parent, children = line.strip().split("->")
        parent = parent.strip()
        children = children.split()
        # Lee las reglas lexicas
        if len(children) == 2:
            if parent not in lexicon_rules:
                lexicon_rules[parent] = set()
            lexicon_rules[parent].add(children[0])
            probabilities[parent,children[0]] = float(children[1])
        # Lee las reglas gramaticales
        else:
            # Crea la regla (parent, (c1, c2)) como en lisp ;)
            rule = (parent, tuple(children[:2]))
            probabilities[rule] = float(children[2])
            gr_temp_rules.append(rule)
    # Crear un diccionario con las reglas invertidas, eso permite buscar al nodo padre
    for (parent, (lc, rc)) in gr_temp_rules:
        # Si no existe la entrada, se crea
        if (lc, rc) not in grammar_rules:
            grammar_rules[lc, rc] = list()
        # Se agrega el caso
        grammar_rules[lc,rc].append(parent)
    return grammar_rules,lexicon_rules,probabilities

## Algoritmos CKY y PCKY

Ahora se implementa el algoritmo CKY, mismo que recibe como entrada una oración en forma de cadena de texto, un diccionario con reglas gramaticales y un lexicón adecuado a la gramática. Opcionalmente, se puede definir el argumento de entrada `verbose` al valor `True`, de esta manera se mostrará en pantalla cada uno de los pasos que sigue el algoritmo para crear la matriz correspondiente. Esta matriz también tendrá forma de diccionario.

In [ ]:
def cky_parser(sentence, grammar, lexicon, verbose=False):
    ''' 
    The function cky_parser implements CKY algorithm.

        Parameters:
            sentence (str): A sentence to be analyzed
            grammar (dic): Dictionary with the grammar rules indexed by children
            lexicon (dic): Dictionary with lexical rules indexed by PoS
            verbose (bool): If it is True, show every step in the algorithm
        Returns:
            matrix_cky (dic): Dictionary with all possible cells written by the algorithm
    '''
    words = sentence.lower().split()
    if verbose:
        print(words)
    matrix_cky = dict()
    for i in range(len(words)):
        # Las ordenadas van del 0 al 4
        ordinate = 0
        if verbose:
            print("0: i=",i)
        # Las abscisas van de 1 a 5
        for abscissa in range(i+1, len(words)+1):
            # Se agrega una celda
            matrix_cky[(ordinate, abscissa)] = list()
            if verbose:
                print("1: abscissa=",abscissa,"ordinate=",ordinate,"a-o=",abscissa-ordinate)
            # Si se opera sobre la diagonal
            if(abscissa-ordinate == 1):
                # Combina la palabra con las categorías del lexicón
                for key in lexicon:
                    if verbose:
                        print("2: key=",key,"word=",words[ordinate])
                    # Si la combinación existe en el lexicón se agrega a la matriz
                    if(words[ordinate] in lexicon[key]):
                        matrix_cky[(ordinate,abscissa)].append(
                            (key,0,words[ordinate],words[ordinate]))
                        if verbose:
                            print("2:",matrix_cky)
            # Si hay que operar sobre dos celdas
            elif(abscissa-ordinate > 1):
                if verbose:
                    print("3:","(",ordinate,",",abscissa,")")
                # Obtener los valores de las celdas
                for index in range(abscissa-ordinate-1):
                    left = matrix_cky[(ordinate,abscissa-1-index)]
                    down = matrix_cky[(abscissa-1-index,abscissa)]
                    if verbose:
                        print("4: index=",index)
                        print("4: left=",left,"\n  ","down=",down)
                    # Si una de las celdas esta vacía no se hace nada
                    if not left or not down:
                        if verbose:
                            print("5: Nothing to do")
                    else:
                        # Combinar los valores de las celdas
                        for a in left:
                            for b in down:
                                # Obtiene el primer elemento de la tupla
                                # Crea la tupla para buscarla en las reglas
                                if((a[0],b[0]) in grammar):
                                    matrix_cky[(ordinate,abscissa)].append(
                                        (grammar[(a[0],b[0])][0],
                                        abscissa-1-index,a[0],b[0]))
                                    if verbose:
                                        print("6: add=",grammar[(a[0],b[0])][0],abscissa-1-index,a[0],b[0])
                                else:
                                    if verbose:
                                        print("6: Nothing to do")
            if verbose:
                print(matrix_cky)
                print("9: Next Ordinate")
            ordinate+=1
    return matrix_cky

Enseguida se crean dos funciones, la primera, `get_prob()`, se usará para obtener las probabilidades del diccionario de probabilidades, la segunda, `update_prob()`, se usará para actualizar las probabilidades desde los nodos intermedios hasta el nodo padre. 

In [ ]:
def get_prob(node, prob_dict):
    '''
    The function get_prob gets a probability from a dictionary.

        Parameters:
            node (tuple): It's a grammar or lexical rule writen in tuple format
            prob_dict (dict): Dictionary with rules as keys an probabilities as values
        Returns:
            prob_dict[node] (float): A probability
    '''
    return prob_dict[node]

def update_prob(left, down, actual, prob_dict):
    '''
    The function get_prob gets a probability from a dictionary.

        Parameters:
            left (tuple): A tuple with information of a left node
            down (tuple): A tuple with information of a right node
            actual (str): Parent node value
            prob_dict (dict): Dictionary with rules as keys an probabilities as values
        Returns:
            prob_dict[node] (float): A probability
    '''
    actual_prob = get_prob((actual,(left[0],down[0])),prob_dict)
    prob = left[4]*down[4]*actual_prob
    return prob

Se actualiza `cky_parser()`. Ahora `pcky_parser()` agregará la información de la probabilidad a cada lista agregada a `matrix_pcky`.

In [ ]:
def pcky_parser(sentence, grammar, lexicon, probabilities_table, verbose=False):
    ''' 
    The function pcky_parser implements PCKY algorithm.

        Parameters:
            sentence (str): A sentence to be analyzed
            grammar (dic): Dictionary with the grammar rules indexed by children
            lexicon (dic): Dictionary with lexical rules indexed by PoS
            probabilities_table (dic): Dictionary with the probability of lexical and gramar rules
            verbose (bool): If it is True, show every step in the algorithm
        Returns:
            matrix_pcky (dic): Dictionary with all possible cells written by the algorithm
    '''
    words = sentence.lower().split()
    if verbose:
        print(words)
    matrix_pcky = dict()
    for i in range(len(words)):
        # Las ordenadas van del 0 al 4
        ordinate = 0
        if verbose:
            print("0: i=",i)
        # Las abscisas van de 1 a 5
        for abscissa in range(i+1,len(words)+1):
            # Se agrega una celda
            matrix_pcky[(ordinate, abscissa)] = list()
            if verbose:
                print("1: abscissa=",abscissa,"ordinate=",ordinate,"a-o=",abscissa-ordinate)
            # Si se opera sobre la diagonal
            if(abscissa-ordinate == 1):
                # Combina la palabra con las categorías del lexicón
                for key in lexicon:
                    if verbose:
                        print("2: key=",key,"word=",words[ordinate])
                    # Si la combinación existe en el lexicón se agrega a la matriz
                    if(words[ordinate] in lexicon[key]):
                        matrix_pcky[(ordinate,abscissa)].append(
                            (key,0,words[ordinate],words[ordinate],
                            get_prob((key,words[ordinate]),probabilities_table)))
                        if verbose:
                            print("2:",matrix_pcky)
            # Si hay que operar sobre dos celdas
            elif(abscissa-ordinate > 1):
                if verbose:
                    print("3:","(",ordinate,",",abscissa,")")
                # Obtener los valores de las celdas
                for index in range(abscissa-ordinate-1):
                    left = matrix_pcky[(ordinate,abscissa-1-index)]
                    down = matrix_pcky[(abscissa-1-index,abscissa)]
                    if verbose:
                        print("4: index=",index)
                        print("4: left=",left,"\n  ","down=",down)
                    # Si una de las celdas esta vacía no se hace nada
                    if not left or not down:
                        if verbose:
                            print("5: Nothing to do")
                    else:
                        # Combinar los valores de las celdas
                        for a in left:
                            for b in down:
                                # Obtiene el primer elemento de la tupla
                                # Crea la tupla para buscarla en las reglas
                                if((a[0],b[0]) in grammar):
                                    matrix_pcky[(ordinate,abscissa)].append(
                                        (grammar[(a[0],b[0])][0],
                                        abscissa-1-index,a[0],b[0],
                                        update_prob(a,b,grammar[(a[0],b[0])][0],probabilities_table))
                                        )
                                    if verbose:
                                        print("6: add=",grammar[(a[0],b[0])][0],abscissa-1-index,a[0],b[0])
                                else:
                                    if verbose:
                                        print("6: Nothing to do")
            if verbose:
                print(matrix_pcky)
                print("9: Next Ordinate")
            ordinate+=1
    return matrix_pcky

## Obtención de resultados

La función `tree()` recorre una matriz generada por el algoritmo CKY en busca de árboles a partir del valor de un nodo en particular, tomando en cuenta una ruta a través de un índice creado por el algoritmo. 

In [ ]:
def tree (matrix, cell, parent="S", index=0, pcky=False):
    '''
    Create a tree from a matrix created by the CKY algorithm.
    See def cky_parser(sentence, grammar, lexicon, verbose=False) function.

        Parameters:
            matrix (dic): Dictionary with CKY algorithm information
            cell (tuple): Node in the matrix to start the analysis
            parent (str): Axiom in the grammar or parent node value for the tree
            index (int): If there is more than one value in cell, this variable let choose which one analyze
            pcky (bool): Enable if matrix contains PCKY algorithm information
        Returns:
            list (list): A list with trees in list shape
    '''
    list = []
    # Revisa que la lista no este vacía
    if len(matrix[cell]) == 0:
        return list
    # Revisa que el índice sea 0, significa que es una hoja
    if matrix[cell][0][1] == 0:
        # Encuentra la hoja que corresponde
        if matrix[cell][index][0] == parent:
            if pcky:
                return [(matrix[cell][index][0],matrix[cell][index][4]),
                         matrix[cell][index][2]]
            else:
                return [matrix[cell][index][0], matrix[cell][index][2]]
        else:
            # Busca en otra hoja
            if index+1 < len(matrix[cell]):
                if pcky:
                    return tree(matrix,cell,parent,index+1,pcky=True)
                else:
                    return tree(matrix,cell,parent,index+1)
            else:
                return list
    # Si no es una hoja, revisa si es la opción correcta
    elif (matrix[cell][index][0]) == parent:
        # Agrega el símbolo
        if pcky:
            list.append((matrix[cell][index][0],matrix[cell][index][4]))
        else:
            list.append(matrix[cell][index][0])
        # Buscar hijos
        child = []
        # Hijo derecho
        if pcky:
            child.append(
                tree(matrix,
                     (cell[0],matrix[cell][index][1]),
                     matrix[cell][index][2],pcky=True))
        else:
            child.append(
                tree(matrix,
                     (cell[0],matrix[cell][index][1]),
                     matrix[cell][index][2]))
        # Hijo izquierdo
        if pcky:
            child.append(
                tree(matrix,
                     (matrix[cell][index][1],cell[1]),
                     matrix[cell][index][3],pcky=True))
        else:
            child.append(
                tree(matrix,
                     (matrix[cell][index][1],cell[1]),
                     matrix[cell][index][3]))
        #Agrega child a la lista
        list.append(child)
        return list
    else:
        if index+1 < len(matrix[cell]):
            if pcky:
                return tree(matrix,cell,parent,index+1,pcky=True)
            else:
                return tree(matrix,cell,parent,index+1)
        else:
            return list

La función `find_solutions()` servirá como _frontend_ para todas las funciones anteriores, brindará la respuesta propuesta por el algoritmo.

In [ ]:
def find_solutions(sentence, grammar, lexicon, axiom, probabilities_table={}, verbose=False):
    '''
    Find all possible solutions of a CKY analysis in a sentence.

        Parameters:
            sentence (str): A sentence to be analyzed
            grammar (dic): Dictionary with the grammar rules indexed by children
            lexicon (dic): Dictionary with lexical rules indexed by PoS
            axiom (str): Axiom in the grammar
            probabilities_table (dic): Dictionary with the probability of lexical and gramar rules
            verbose (bool): If it is True, show every step in the algorithm

        Returns:
            solutions (list): List with all possible solutinos suggested by the algorithm
            df (dataframe): Data frame with the CKY matrix, useful for printing
    '''
    N = len(sentence.split())
    solutions = list()
    if probabilities_table:
        matrix = pcky_parser(sentence,grammar,lexicon,probabilities_table,verbose)
        candidates = list()
        # Busca en (0,N) aquellas listas que contengan el axioma
        for i in enumerate(matrix[(0,N)]):
            if i[1][0] == axiom:
                # En candidates se guarda el índice y la probabilidad
                candidates.append([i[0],i[1][4]])
        # Selecciona aquel con mayor probabilidad
        candidates = max(candidates,key=itemgetter(1))
        solutions = tree(matrix,(0,N),index=candidates[0],pcky=True)
    else:
        matrix = cky_parser(sentence,grammar,lexicon,verbose)
        for i in enumerate(matrix[(0,N)]):
            if i[1][0] == axiom:
                solutions.append(tree(matrix,(0,N),index=i[0]))
    # Crea un dataframe, útil para ver la matriz
    df = pd.DataFrame(columns = [i for i in range(0,N)],
                        index = [i for i in range(1,N+1)])
    for i in range(0,5):
        for j in range(0,5-i):
            df.loc[j+i+1,i] = matrix[(i,j+i+1)]
    df = df.transpose()
    return solutions, df

In [ ]:
grammar_rules, lexicon_rules, probabilities = read_rules(grammar_file)

In [ ]:
options, matrix = find_solutions("Time flies like an arrow",
                        grammar_rules,lexicon_rules,"S")
pprint(options)
matrix

[['S',
  [['NP', [['Nominal', 'time'], ['Nominal', 'flies']]],
   ['VP', [['Verb', 'like'], ['NP', [['Det', 'an'], ['Nominal', 'arrow']]]]]]],
 ['S',
  [['NP', 'time'],
   ['VP',
    [['Verb', 'flies'],
     ['PP',
      [['Preposition', 'like'],
       ['NP', [['Det', 'an'], ['Nominal', 'arrow']]]]]]]]]]


,1,2,3,4,5
0,"[(NP, 0, time, time), (Nominal, 0, time, time)...","[(S, 1, NP, VP), (NP, 1, Nominal, Nominal), (N...","[(S, 2, NP, VP)]",[],"[(S, 2, NP, VP), (Nominal, 2, Nominal, PP), (S..."
1,NaN,"[(NP, 0, flies, flies), (Nominal, 0, flies, fl...","[(S, 2, NP, VP)]",[],"[(S, 2, NP, VP), (Nominal, 2, Nominal, PP), (V..."
2,NaN,NaN,"[(VP, 0, like, like), (Verb, 0, like, like), (...",[],"[(VP, 3, Verb, NP), (PP, 3, Preposition, NP)]"
3,NaN,NaN,NaN,"[(Det, 0, an, an)]","[(NP, 4, Det, Nominal)]"
4,NaN,NaN,NaN,NaN,"[(NP, 0, arrow, arrow), (Nominal, 0, arrow, ar..."


In [ ]:
options, matrix = find_solutions("Time flies like an arrow",
                        grammar_rules,lexicon_rules,"S",probabilities)
pprint(options)
matrix

[('S', 9.600000000000002e-13),
 [[('NP', 0.002), 'time'],
  [('VP', 6.000000000000001e-10),
   [[('Verb', 0.02), 'flies'],
    [('PP', 1.5000000000000002e-07),
     [[('Preposition', 0.05), 'like'],
      [('NP', 3e-05),
       [[('Det', 0.05), 'an'], [('Nominal', 0.002), 'arrow']]]]]]]]]


,1,2,3,4,5
0,"[(NP, 0, time, time, 0.002), (Nominal, 0, time...","[(S, 1, NP, VP, 1.28e-05), (NP, 1, Nominal, No...","[(S, 2, NP, VP, 5.1200000000000005e-09)]",[],"[(S, 2, NP, VP, 1.1520000000000003e-13), (Nomi..."
1,NaN,"[(NP, 0, flies, flies, 0.002), (Nominal, 0, fl...","[(S, 2, NP, VP, 1.28e-05)]",[],"[(S, 2, NP, VP, 2.8800000000000004e-10), (Nomi..."
2,NaN,NaN,"[(VP, 0, like, like, 0.008), (Verb, 0, like, l...",[],"[(VP, 3, Verb, NP, 1.8000000000000002e-07), (P..."
3,NaN,NaN,NaN,"[(Det, 0, an, an, 0.05)]","[(NP, 4, Det, Nominal, 3e-05)]"
4,NaN,NaN,NaN,NaN,"[(NP, 0, arrow, arrow, 0.002), (Nominal, 0, ar..."


## Cuestionario

1. ¿Es correcto el análisis sintáctico que se ha obtenido? Justifica la respuesta.

Sí. Basado en la gramática proporcionada el resultado es satisfactorio y se muestra a continuación.

\usepackage{tikz}
\usepackage{tikz-qtree}
\begin{figure}[!ht]
    \centering
        \Tree [.S~(9.600000000000002e-13)
                [.NP~(0.002) time ]
                [.VP 
                    [.Verb~(0.02) flies ]
                    [.PP~(1.5000000000000002e-07) 
                        [.Preposition~(0.05) like ]
                        [.NP~(3e-05) 
                            [.Det~(0.05) an ]
                            [.Nominal~(0.002) arrow ] ] ] ] ]
\caption{Árbol sintáctico proporcionado por el algoritmo PCKY}
\end{figure}

2. ¿Cuáles son las limitaciones de aplicar el algoritmo CKY probabilístico para realizar el análisis sintáctico? Justifica la respuesta.

Los problemas se presentan debido a la información guardada en la gramática, si ésta no tiene un elemento particular necesario para el análisis, proporcionará resultados truncos.

Por supuesto, las probabilidades de aparición de la gramática podrían llevar a resultados dudosos, incluso cuando su objetivo es el contrario, pero estas probabilidades siempre dependerán del corpus de origen, que puede ser distinto al de la oración a analizar, ya sea en registro, en dialecto, etc.

En el caso particular de este ejercicio, el hecho de tener algunos símbolos intermedios creados para llegar a la forma normal de Chomsky permite que el algoritmo genere posibles soluciones impensables para el análisis linguístico. El algoritmo CKY propone dos soluciones y el PCKY seleccionó el correcto, pero si se examina la opción desechada se puede observar:

    ['NP', [['Nominal', 'time'], ['Nominal', 'flies']]]

Lo cual es un elemento impensable pues sugiere que una frase nomimal puede tener dos núcleos. Este análisis va en contra del principio de endocentrismo que establece, como su nombre indica, que cada frase tiene un núcleo que determina el tipo de la frase.

Finalmente, el algoritmo que usa este tipo de gramáticas, arrastra la misma gran crítica que los lingǘistas hacen a la teoría de Chomsky: No es capaz de analizar lenguas aglutinantes. 

3. ¿Qué posibles mejoras que se podrían aplicar para mejorar el rendimiento del análisis sintáctico? Justifica la respuesta.

El algoritmo se basa en el uso de cualquier gramática en forma normal de Chomsky, esto es porque computacionalmente es posible generar automáticamente generadas mediante corpus de análisis, sin embargo, el programa minimalista de Chomsky sugiere la teoría X-barra que tiene su principio en los estudios de Ray Jackendoff. Esta teoría proporciona la posibilidad de crear gramáticas consistentes siempre en la forma normal de Chomsky. Dejar de ignorar el conocimiento linguístico proporcionaría mejores gramáticas para la IA.

Por otra parte, analizar solo las probabilidades de los subárboles que llevan a una propuesta de solución aumentaría el rendimiento a nivel de cómputo.

 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=16bbac0e-c0cf-475a-9977-7067c93e2eaf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>